Connected to py39 (Python 3.9.19)

In [1]:
import torch
import numpy as np

In [2]:
import sys
import os

# Add the current file's directory to the Python path
current_dir = os.path.dirname(os.path.abspath('..'))
if current_dir not in sys.path:
    sys.path.append(current_dir)

from solution import *

In [3]:
# Construct the full range of bfloat16
y = torch.arange(0, 2**16, dtype=torch.int32)
y = bits_to_bfloat16(y)
print(y.shape, y)

torch.Size([65536]) tensor([0.0000e+00, 9.1835e-41, 1.8367e-40,  ...,        nan,        nan,
               nan], dtype=torch.bfloat16)


In [4]:
z = bfloat16_to_bits(y)
print(z.shape, z)

torch.Size([65536]) tensor([    0,     1,     2,  ..., 65533, 65534, 65535], dtype=torch.int32)


In [5]:
# Construct the full range of uint8
x = torch.arange(0, 2**8, dtype=torch.int32)
print(x.shape, x)

torch.Size([256]) tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175

In [6]:
print(*decompose_16bit(z))
print(*decompose_8bit_e4m3(x))
print(*decompose_8bit_e5m2(x))

tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.int32) tensor([  0,   0,   0,  ..., 255, 255, 255], dtype=torch.int32) tensor([  0,   1,   2,  ..., 125, 126, 127], dtype=torch.int32)
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 

In [7]:
sign, exponent, mantissa = decompose_16bit(z)
bits = compose_16bit(sign, exponent, mantissa)
print(torch.equal(bits, z))

True


In [8]:
# Test that you can convert bfloat16 to bits and back
y_ = bits_to_bfloat16(
    bfloat16_to_bits(y)
)
comparison = torch.isclose(
        y,
        y_
)
comparison[torch.isnan(y) & torch.isnan(y_)] = True
comparison.all()

tensor(True)

In [9]:
# Test that you can convert bfloat16 to bits, decompose, recompose and convert back
y_bits = bfloat16_to_bits(y)
components = decompose_16bit(y_bits)
y_bits_ = compose_16bit(*components)
y_ = bits_to_bfloat16(y_bits_)

comparison = torch.isclose(
        y,
        y_
)
comparison[torch.isnan(y) & torch.isnan(y_)] = True
comparison.all()

tensor(True)

In [10]:
# Construct infinity
s = torch.tensor([0], dtype=torch.int32)
e = torch.tensor([0xFF], dtype=torch.int32)
m = torch.tensor([0], dtype=torch.int32)
n = bits_to_bfloat16(compose_16bit(s, e, m))
print(n)

tensor([inf], dtype=torch.bfloat16)


In [11]:
# Construct nan
s = torch.tensor([0], dtype=torch.int32)
e = torch.tensor([0xFF], dtype=torch.int32)
m = torch.tensor([1], dtype=torch.int32)
n = bits_to_bfloat16(compose_16bit(s, e, m))
print(n)

tensor([nan], dtype=torch.bfloat16)


In [12]:
# Test bfloat16 to e5m2 and back
t = torch.tensor([0.3125], dtype=torch.bfloat16)
t_bits = bfloat16_to_bits(t)

encoded = encode_as_e5m2(t_bits)
decoded = decode_from_e5m2(encoded)
t_quantized = bits_to_bfloat16(decoded)
print(t, t_quantized)

tensor([0.3125], dtype=torch.bfloat16) tensor([0.3125], dtype=torch.bfloat16)


In [13]:
# Test rounding up when converting from bfloat16 to e5m2
s = torch.tensor([0], dtype=torch.int32)
e = torch.tensor([0b01111100], dtype=torch.int32)
m = torch.tensor([0b1011111], dtype=torch.int32)  # Changed to 1011111 to test rounding up
t_bits = compose_16bit(s, e, m)
t = bits_to_bfloat16(t_bits)  # This is 0 01111100 1011111 in binary

encoded = encode_as_e5m2_round_up(t_bits)
decoded = decode_from_e5m2(encoded)
t_quantized = bits_to_bfloat16(decoded)

# Construct the rounded up e5m2 value
encoded_e5m2 = encode_as_e5m2(t_bits)
s, e, m = decompose_8bit_e5m2(encoded_e5m2)

# Manually round up (without worrying about mantissa overflow)
m += 1

# Compose the rounded e5m2 value
rounded_e5m2 = compose_e5m2(s, e, m)

# Convert back to bfloat16 using decode_from_e5m2
decoded_bits = decode_from_e5m2(rounded_e5m2)
t_expected = bits_to_bfloat16(decoded_bits)

print("Original:", t)
print("Quantized:", t_quantized)
print("Rounded up correctly:", torch.isclose(t_quantized, t_expected))

Original: tensor([0.2178], dtype=torch.bfloat16)
Quantized: tensor([0.2188], dtype=torch.bfloat16)
Rounded up correctly: tensor([True])


In [14]:
# Test encode_as_e5m2 and decode_as_e5m2 for all possible e5m2 values

# Generate all possible 8-bit values (0 to 255)
all_e5m2_values = torch.arange(0, 256, dtype=torch.uint8)

# Convert to int32 for compatibility with our functions
all_e5m2_int32 = all_e5m2_values.to(dtype=torch.int32)

# Decode all e5m2 values to bfloat16 bits
decoded_bits = decode_from_e5m2(all_e5m2_int32)

# Convert to bfloat16
bfloat16_values = bits_to_bfloat16(decoded_bits)

# Encode back to e5m2
encoded_e5m2 = encode_as_e5m2(bfloat16_to_bits(bfloat16_values))

# Check if the encoding-decoding process is reversible
is_reversible = torch.all(encoded_e5m2 == all_e5m2_int32)

print(f"Encoding-decoding process is reversible for all e5m2 values: {is_reversible}")

# Additional checks
if not is_reversible:
    # Find where the mismatch occurs
    mismatch = encoded_e5m2 != all_e5m2_int32
    mismatch_indices = torch.where(mismatch)[0]
    print(f"Mismatches found at indices: {mismatch_indices}")
    
    # Print a few mismatched values for inspection
    for idx in mismatch_indices[:5]:  # Show first 5 mismatches
        original = all_e5m2_int32[idx].item()
        encoded = encoded_e5m2[idx].item()
        print(f"Index {idx}: Original {original:08b}, Encoded {encoded:08b}")



Encoding-decoding process is reversible for all e5m2 values: True


In [15]:
# Test round_to_fp8_represented_as_int8() and undo_int8_fp8() for all bfloat16 values

# Generate all possible 16-bit values (0 to 65535)
all_bfloat16_values = torch.arange(0, 2**16, dtype=torch.int32)

# Convert to bfloat16
bfloat16_values = bits_to_bfloat16(all_bfloat16_values)

# Round to fp8 (e5m2)
fp8_values = round_to_fp8_represented_as_int8(bfloat16_values, n_mantissa=2)

# Convert back to bfloat16
recovered_bfloat16 = undo_int8_fp8(fp8_values, n_mantissa=2)

# Create a mask for non-NaN values
non_nan_mask = ~torch.isnan(bfloat16_values) & ~torch.isnan(recovered_bfloat16)

# Check if the rounding process preserves order (ignoring NaNs)
is_order_preserved = torch.all(torch.argsort(bfloat16_values[non_nan_mask]) == torch.argsort(recovered_bfloat16[non_nan_mask]))

print(f"Rounding process preserves order (ignoring NaNs): {is_order_preserved}")


Rounding process preserves order (ignoring NaNs): False


In [16]:
print(recovered_bfloat16[non_nan_mask])

tensor([ 2.,  2.,  2.,  ..., -1., -1., -1.], dtype=torch.bfloat16)


In [17]:

# Check for special cases
inf_mask = torch.isinf(bfloat16_values)
nan_mask = torch.isnan(bfloat16_values)
zero_mask = bfloat16_values == 0

print(f"Inf values preserved: {torch.all(torch.isinf(recovered_bfloat16[inf_mask]))}")
print(f"NaN values preserved: {torch.all(torch.isnan(recovered_bfloat16[nan_mask]))}")
print(f"Zero values preserved: {torch.all(recovered_bfloat16[zero_mask] == 0)}")

# Check if values are close within fp8 precision (ignoring NaNs)
non_special_mask = ~(inf_mask | nan_mask | zero_mask)
is_close = torch.isclose(bfloat16_values[non_special_mask], recovered_bfloat16[non_special_mask], rtol=1e-2, atol=1e-2)
print(f"Non-special values are close (ignoring NaNs): {torch.all(is_close)}")

# If not all values are close, print some statistics
if not torch.all(is_close):
    not_close = ~is_close
    print(f"Number of values not close: {torch.sum(not_close)}")
    print(f"Max absolute difference: {torch.max(torch.abs(bfloat16_values[non_special_mask][not_close] - recovered_bfloat16[non_special_mask][not_close]))}")
    print(f"Mean absolute difference: {torch.mean(torch.abs(bfloat16_values[non_special_mask][not_close] - recovered_bfloat16[non_special_mask][not_close]))}")

# Test for stochastic rounding
num_rounds = 1000
stochastic_results = torch.zeros_like(bfloat16_values)

for _ in range(num_rounds):
    fp8_values = round_to_fp8_represented_as_int8(bfloat16_values, n_mantissa=2)
    recovered = undo_int8_fp8(fp8_values, n_mantissa=2)
    stochastic_results += recovered

average_result = stochastic_results / num_rounds

# Check if the average result is close to the original values (ignoring NaNs)
non_nan_mask = ~torch.isnan(bfloat16_values) & ~torch.isnan(average_result)
is_average_close = torch.isclose(bfloat16_values[non_nan_mask], average_result[non_nan_mask], rtol=1e-2, atol=1e-2)
print(f"Average stochastic result is close to original (ignoring NaNs): {torch.all(is_average_close)}")

# If not all average values are close, print some statistics
if not torch.all(is_average_close):
    not_close = ~is_average_close
    print(f"Number of average values not close: {torch.sum(not_close)}")
    print(f"Max absolute difference in averages: {torch.max(torch.abs(bfloat16_values[non_nan_mask][not_close] - average_result[non_nan_mask][not_close]))}")
    print(f"Mean absolute difference in averages: {torch.mean(torch.abs(bfloat16_values[non_nan_mask][not_close] - average_result[non_nan_mask][not_close]))}")


Inf values preserved: False
NaN values preserved: False
Zero values preserved: False
Non-special values are close (ignoring NaNs): False
Number of values not close: 55024
Max absolute difference: 3.3895313892515355e+38
Mean absolute difference: inf
Average stochastic result is close to original (ignoring NaNs): False
Number of average values not close: 55301
Max absolute difference in averages: inf
Mean absolute difference in averages: inf


At this point we are able to:
- Construct the full range of bfloat16, uint8
- Convert bfloat16 to bits and back
- Convert bfloat16 to sign, exponent, mantissa and back
- Test that conversion is correct except NaN values
- Truncate when bfloat16 to e5m2 with mantissa chopping
- Round up when converting from bfloat16 to e5m2

Next:
- Handle subnormal, inf, nan special cases in encode and decode 
- Round by stochastic rounding scheme